# Read KFF Data Files

This notebook reads all data files under `2433_p3_data/KFF_data/` directory and displays the first 5 rows of each file.

KFF (Kaiser Family Foundation) data files contain healthcare-related statistics and information.

In [ ]:
# 导入必要的库
import pandas as pd
from pathlib import Path
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# 设置pandas显示选项
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 50)

In [ ]:
# 定义KFF数据目录
KFF_DATA_DIR = Path('2433_p3_data/KFF_data')

# 检查目录是否存在
if not KFF_DATA_DIR.exists():
    print(f"错误: 目录 {KFF_DATA_DIR} 不存在!")
else:
    print(f"✓ 找到数据目录: {KFF_DATA_DIR}")
    print(f"  绝对路径: {KFF_DATA_DIR.absolute()}")

In [ ]:
# 查找所有数据文件
data_files = sorted(list(KFF_DATA_DIR.glob('*.csv')))

print(f"发现 {len(data_files)} 个CSV文件:")
for i, file in enumerate(data_files, 1):
    file_size = file.stat().st_size / 1024  # KB
    print(f"  {i}. {file.name} ({file_size:.2f} KB)")

## 读取所有KFF数据文件

In [ ]:
# 读取所有CSV文件到字典中
kff_data = {}
errors = {}

for file_path in data_files:
    file_name = file_path.name
    try:
        print(f"正在读取: {file_name}...", end=" ")
        df = pd.read_csv(file_path, low_memory=False)
        kff_data[file_name] = df
        print(f"✓ 成功 (形状: {df.shape})")
    except Exception as e:
        errors[file_name] = str(e)
        print(f"✗ 失败: {e}")

print(f"\n总计: 成功读取 {len(kff_data)} 个文件, 失败 {len(errors)} 个文件")

## 显示每个文件的前5行

In [ ]:
# 显示每个文件的前5行
for i, (file_name, df) in enumerate(sorted(kff_data.items()), 1):
    print("="*100)
    print(f"文件 {i}/{len(kff_data)}: {file_name}")
    print(f"形状: {df.shape[0]:,} 行 × {df.shape[1]} 列")
    print("="*100)
    
    # 显示列名
    print(f"\n列名 ({len(df.columns)} 列):")
    print(list(df.columns))
    
    # 显示前5行
    print("\n前5行数据:")
    display(df.head())
    
    # 显示基本统计信息
    print("\n数据类型:")
    print(df.dtypes)
    
    print("\n缺失值统计:")
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("没有缺失值")
    
    print("\n" + "-"*100 + "\n")

## 数据概览汇总

In [ ]:
# 创建数据概览汇总表
summary_data = []

for file_name, df in sorted(kff_data.items()):
    summary_data.append({
        '文件名': file_name,
        '行数': df.shape[0],
        '列数': df.shape[1],
        '总缺失值': df.isnull().sum().sum(),
        '缺失率(%)': f"{(df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100):.2f}%",
        '内存使用(MB)': f"{df.memory_usage(deep=True).sum() / 1024 / 1024:.2f}"
    })

summary_df = pd.DataFrame(summary_data)

print("="*100)
print("KFF数据文件汇总")
print("="*100)
display(summary_df)

print(f"\n总行数: {summary_df['行数'].sum():,}")
print(f"平均列数: {summary_df['列数'].mean():.1f}")

## 检查数据一致性

In [ ]:
# 检查所有文件是否有相同的列结构
if len(kff_data) > 0:
    first_file = list(kff_data.keys())[0]
    first_columns = set(kff_data[first_file].columns)
    
    print("列结构一致性检查:")
    all_same = True
    
    for file_name, df in kff_data.items():
        current_columns = set(df.columns)
        if current_columns == first_columns:
            print(f"  ✓ {file_name}: 列结构一致")
        else:
            all_same = False
            print(f"  ✗ {file_name}: 列结构不同")
            missing = first_columns - current_columns
            extra = current_columns - first_columns
            if missing:
                print(f"    缺少的列: {missing}")
            if extra:
                print(f"    额外的列: {extra}")
    
    if all_same:
        print("\n✓ 所有文件的列结构一致，可以安全地合并!")
    else:
        print("\n⚠ 文件列结构不一致，合并前需要进行对齐处理。")

## 可选: 合并所有年份的数据

In [ ]:
# 如果所有文件列结构一致，可以合并它们
try:
    combined_df = pd.concat(kff_data.values(), ignore_index=True)
    print(f"✓ 成功合并所有文件")
    print(f"  合并后形状: {combined_df.shape[0]:,} 行 × {combined_df.shape[1]} 列")
    
    # 如果有年份列，显示年份分布
    year_cols = [col for col in combined_df.columns if 'year' in col.lower()]
    if year_cols:
        print(f"\n年份分布 (基于列 '{year_cols[0]}'):")
        print(combined_df[year_cols[0]].value_counts().sort_index())
    
    # 将合并后的数据存储到全局变量
    globals()['kff_combined'] = combined_df
    print("\n✓ 合并数据已保存到变量 'kff_combined'")
    
except Exception as e:
    print(f"✗ 合并失败: {e}")
    print("  文件列结构可能不一致，请先检查列结构。")

## 数据导出 (可选)

In [ ]:
# 可选: 将前5行数据导出为单独的预览文件
preview_dir = Path('2433_p3_data/KFF_data/previews')
preview_dir.mkdir(parents=True, exist_ok=True)

print("保存前5行预览文件...")
for file_name, df in kff_data.items():
    preview_file = preview_dir / f"{file_name.replace('.csv', '_preview.csv')}"
    df.head().to_csv(preview_file, index=False)
    print(f"  ✓ {preview_file.name}")

print(f"\n✓ 所有预览文件已保存到: {preview_dir}")

In [ ]:
# 将所有数据保存到全局变量以便后续使用
globals()['kff_data'] = kff_data
print(f"✓ 所有KFF数据已保存到变量 'kff_data' (字典类型)")
print(f"  可用文件: {list(kff_data.keys())}")
print(f"\n使用示例:")
print("  - 访问2020年数据: kff_data['raw_data_2020.csv']")
print("  - 访问合并数据: kff_combined (如果已执行合并)")